# Technical Factor - Psychological Line (PSY)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        m_avg(m_sum(IF(close > m_lag(close, 1), 1, 0), 12), 6) AS PSY,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(PSY > 50 AND m_lag(PSY, 1) <= 50, 1, 0) AS TRBY1,
        IF(PSY < 50 AND m_lag(PSY, 1) >= 50, 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(PSY > m_lag(PSY, 1) AND m_lag(PSY, 1) >= m_lag(PSY, 2), 1, 0) AS MTBY1,
        IF(PSY < m_lag(PSY, 1) AND m_lag(PSY, 1) <= m_lag(PSY, 2), 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(PSY < 25 AND (PSY - m_lag(PSY, 1)) > 0, 1, 0) AS RVBY1,
        IF(PSY > 75 AND (PSY - m_lag(PSY, 1)) < 0, 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(PSY, 1) <= 75 AND PSY > 75, 1, 0) AS BKBY1,
        IF(m_lag(PSY, 1) >= 25 AND PSY < 25, 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        PSY,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,PSY,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-01-24,600519.SH,6.666667,0,0,0,0,0,0,0,0
1,2025-01-27,600519.SH,6.500000,0,0,0,0,0,0,0,0
2,2025-02-05,600519.SH,6.166667,0,0,0,1,0,0,0,0
3,2025-02-06,600519.SH,5.833333,0,0,0,1,0,0,0,0
4,2025-02-07,600519.SH,5.500000,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
222,2025-12-25,600519.SH,6.500000,0,0,1,0,1,0,0,0
223,2025-12-26,600519.SH,6.500000,0,0,0,0,0,0,0,0
224,2025-12-29,600519.SH,6.166667,0,0,0,1,0,0,0,0
225,2025-12-30,600519.SH,5.666667,0,0,0,1,0,0,0,0
